In [36]:
import pysam
import numpy as np
import pandas as pd
from collections import defaultdict
import random
import rich_click as click
from tqdm.auto import tqdm
from icecream import ic
def reverse_complement(seq):
    """
    Return the reverse complement of a DNA sequence.
    """
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join([complement[base] for base in reversed(seq)])

# Function to extract CpG positions and methylation states from a read
def extract_cpg_states(read, reference_seq, read_start):
    """
    Extract methylation states (0 = unmethylated, 1 = methylated) for CpG sites in a read.
    Accounts for bisulfite conversion, insertions, deletions, and reverse strand alignment.
    """
    # Skip reads with soft clips
    if any(op[0] == 4 for op in read.cigartuples):
        return {}

    # Handle reverse strand reads
    if read.flag in [83, 163, 1]:
        seq = reverse_complement(read.query_sequence)
    else:
        seq = read.query_sequence

    ref_positions = read.get_reference_positions(full_length=True)
    cigar = read.cigartuples
    states = {}

    # Iterate through the read sequence and reference positions
    read_index = 0
    ref_index = read.reference_start
    for op, length in cigar:
        if op == 0:  # Match
            for i in range(length):
                if ref_positions[read_index] is not None:
                    # Calculate the position in the fetched reference sequence
                    ref_seq_index = ref_index - read_start
                    if ref_seq_index >= 0 and ref_seq_index + 1 < len(reference_seq):
                        # Check if the current position is part of a CpG site
                        if reference_seq[ref_seq_index].upper() == "C" and reference_seq[ref_seq_index + 1].upper() == "G":
                            ic(i)
                            # ic(ref_positions)
                            ic(ref_seq_index)
                            ic(seq[read_index].upper())
                            # Determine methylation state
                            if seq[read_index].upper() == 'C':
                                states[ref_index] = 1  # Methylated
                            elif seq[read_index].upper() == 'T':
                                states[ref_index] = 0  # Unmethylated
                read_index += 1
                ref_index += 1
        elif op == 1:  # Insertion
            read_index += length
        elif op == 2:  # Deletion
            ref_index += length
        elif op == 4:  # Soft clip (already skipped)
            pass

    return states

# Function to build the CpG transition and methylation table
def build_cpg_table(bam_file, reference_genome):
    """
    Build a table for CpG sites with transition counts and methylation counts using pandas.
    """
    # Initialize the table as a pandas DataFrame
    cpg_table = pd.DataFrame(columns=[
        "chromosome", "start", "end", "0->0", "0->1", "1->0", "1->1", "0", "1"
    ]).set_index(["chromosome", "start", "end"])

    with pysam.AlignmentFile(bam_file, "rb") as bam:
        for read in tqdm(bam):
            if read.is_unmapped:
                continue
            # Fetch the reference sequence for the region of the read
            chromosome = read.reference_name
            start = read.reference_start
            end = read.reference_end
            reference_seq = reference_genome.fetch(chromosome, start, end)
            states = extract_cpg_states(read, reference_seq, start)
            sorted_positions = sorted(states.keys())
            # Update the table
            for i in range(1, len(sorted_positions)):
                prev_pos = sorted_positions[i - 1]
                curr_pos = sorted_positions[i]
                prev_state = states[prev_pos]
                curr_state = states[curr_pos]
                # Update transition counts
                transition_key = f"{prev_state}->{curr_state}"
                if (chromosome, curr_pos, curr_pos + 1) not in cpg_table.index:
                    cpg_table.loc[(chromosome, curr_pos, curr_pos + 1)] = [0, 0, 0, 0, 0, 0]
                cpg_table.at[(chromosome, curr_pos, curr_pos + 1), transition_key] += 1
            # Update methylation counts
            for pos, state in states.items():
                if (chromosome, pos, pos + 1) not in cpg_table.index:
                    cpg_table.loc[(chromosome, pos, pos + 1)] = [0, 0, 0, 0, 0, 0]
                cpg_table.at[(chromosome, pos, pos + 1), str(state)] += 1

    return cpg_table

# Function to simulate long reads based on the CpG table
def simulate_long_reads(cpg_table, read_length, num_reads, output_bam):
    """
    Simulate long reads using the CpG table and save to a BAM file.
    """
    # Convert the table to a list of CpG sites
    cpg_sites = cpg_table.index.tolist()

    # Initialize BAM file for writing
    with pysam.AlignmentFile(output_bam, "wb", header=pysam.AlignmentHeader.from_references([chromosome for chromosome, _, _ in cpg_sites], [max(end for _, _, end in cpg_sites) + read_length])) as out_bam:
        for read_id in tqdm(range(num_reads)):
            # Randomly select a starting CpG site
            chromosome, start, end = random.choice(cpg_sites)
            # Initialize the read
            states = {}
            prev_state = None
            pos = start
            while pos < start + read_length:
                if (chromosome, pos, pos + 1) not in cpg_table.index:
                    break
                # Check if the CpG site has any coverage
                if cpg_table.loc[(chromosome, pos, pos + 1), "0"] + cpg_table.loc[(chromosome, pos, pos + 1), "1"] == 0:
                    break
                # Determine the methylation state
                if prev_state is None:
                    # Use the beta value for the first CpG
                    beta = cpg_table.loc[(chromosome, pos, pos + 1), "1"] / (cpg_table.loc[(chromosome, pos, pos + 1), "0"] + cpg_table.loc[(chromosome, pos, pos + 1), "1"])
                    states[pos] = 1 if random.random() < beta else 0
                else:
                    # Use the transition counts if available
                    transition_counts = [
                        cpg_table.loc[(chromosome, pos, pos + 1), "0->0"],
                        cpg_table.loc[(chromosome, pos, pos + 1), "0->1"],
                        cpg_table.loc[(chromosome, pos, pos + 1), "1->0"],
                        cpg_table.loc[(chromosome, pos, pos + 1), "1->1"]
                    ]
                    if sum(transition_counts) == 0:
                        # Use the beta value if no transition data
                        beta = cpg_table.loc[(chromosome, pos, pos + 1), "1"] / (cpg_table.loc[(chromosome, pos, pos + 1), "0"] + cpg_table.loc[(chromosome, pos, pos + 1), "1"])
                        states[pos] = 1 if random.random() < beta else 0
                    else:
                        # Use the transition probabilities
                        if prev_state == 0:
                            if cpg_table.loc[(chromosome, pos, pos + 1), "0->0"] + cpg_table.loc[(chromosome, pos, pos + 1), "0->1"] == 0:
                                break
                            states[pos] = 1 if random.random() < cpg_table.loc[(chromosome, pos, pos + 1), "0->1"] / (cpg_table.loc[(chromosome, pos, pos + 1), "0->0"] + cpg_table.loc[(chromosome, pos, pos + 1), "0->1"]) else 0
                        else:
                            if cpg_table.loc[(chromosome, pos, pos + 1), "1->0"] + cpg_table.loc[(chromosome, pos, pos + 1), "1->1"] == 0:
                                break
                            states[pos] = 1 if random.random() < cpg_table.loc[(chromosome, pos, pos + 1), "1->1"] / (cpg_table.loc[(chromosome, pos, pos + 1), "1->0"] + cpg_table.loc[(chromosome, pos, pos + 1), "1->1"]) else 0
                prev_state = states[pos]
                pos += 1
            # Convert states to sequence (C = methylated, T = unmethylated)
            seq = ''.join(['C' if states.get(p, 0) == 1 else 'T' for p in range(start, start + read_length)])
            # Create a read
            read = pysam.AlignedSegment()
            read.query_name = f"simulated_read_{read_id}"
            read.query_sequence = seq
            read.reference_id = out_bam.get_tid(chromosome)  # Chromosome index
            read.reference_start = start
            read.cigar = [(0, read_length)]  # Assume no indels
            read.mapping_quality = 60
            # Add read to BAM file
            out_bam.write(read)

In [8]:
reference_genome = '/gpfs3/chengqiyi_pkuhpc/limingyang/hg38/hg38_only_chromsomes.fa'
reference_genome = pysam.FastaFile(reference_genome)
bam_file = 'SRR2074689_liver_tumor.clean.bam_DMRed.rmdup_intersect.bam'

In [50]:
cpg_table = pd.DataFrame(columns=[
    "chromosome", "start", "end", "0->0", "0->1", "1->0", "1->1", "0", "1"
]).set_index(["chromosome", "start", "end"])

In [51]:
bam = pysam.AlignmentFile(bam_file, "rb")

In [56]:
read = bam.__next__()
read

In [57]:
chromosome = read.reference_name
start = read.reference_start
end = read.reference_end
reference_seq = reference_genome.fetch(chromosome, start, end)
states = extract_cpg_states(read, reference_seq, start)
sorted_positions = sorted(states.keys())
# Update the table
for i in range(1, len(sorted_positions)):
    prev_pos = sorted_positions[i - 1]
    curr_pos = sorted_positions[i]
    prev_state = states[prev_pos]
    curr_state = states[curr_pos]
    # Update transition counts
    transition_key = f"{prev_state}->{curr_state}"
    if (chromosome, curr_pos, curr_pos + 1) not in cpg_table.index:
        cpg_table.loc[(chromosome, curr_pos, curr_pos + 1)] = [0, 0, 0, 0, 0, 0]
    cpg_table.at[(chromosome, curr_pos, curr_pos + 1), transition_key] += 1
# Update methylation counts
for pos, state in states.items():
    if (chromosome, pos, pos + 1) not in cpg_table.index:
        cpg_table.loc[(chromosome, pos, pos + 1)] = [0, 0, 0, 0, 0, 0]
    cpg_table.at[(chromosome, pos, pos + 1), str(state)] += 1

ic| i: 3
ic| ref_seq_index: 3
ic| seq[read_index].upper(): 'T'
ic| i: 20
ic| ref_seq_index: 20
ic| seq[read_index].upper(): 'T'
ic| i: 25
ic| ref_seq_index: 25
ic| seq[read_index].upper(): 'T'
ic| i: 32
ic| ref_seq_index: 32
ic| seq[read_index].upper(): 'T'
ic| i: 34
ic| ref_seq_index: 34
ic| seq[read_index].upper(): 'T'
ic| i: 38
ic| ref_seq_index: 38
ic| seq[read_index].upper(): 'T'
ic| i: 68
ic| ref_seq_index: 68
ic| seq[read_index].upper(): 'T'
ic| i: 74
ic| ref_seq_index: 74
ic| seq[read_index].upper(): 'T'


In [41]:
read.flag

147

In [42]:
read.query_sequence

'TTTTGGTAGTTTTTGATTTTTGGTTTGGTTTTTGTGGTTGGGGAGGGTTGGATTTTATATTTTGTTTTTGAGGTTGTATTATTTATTTAGTTT'

In [43]:
reference_seq

'CCCCGGCAGCCTCTGACTCCCGGCTCGGCTCCCGCGGTCGGGGAGGGTTGGATTTCACACTTTGTTCTCGAGGCCGCACCATTGATTAAGCCC'

In [44]:
states

{1164710: 0,
 1164727: 0,
 1164732: 0,
 1164739: 0,
 1164741: 0,
 1164745: 0,
 1164775: 0,
 1164781: 0}

In [45]:
read.cigartuples

[(0, 93)]

In [46]:
ref_positions = read.get_reference_positions(full_length=True)
ref_positions

[1164707,
 1164708,
 1164709,
 1164710,
 1164711,
 1164712,
 1164713,
 1164714,
 1164715,
 1164716,
 1164717,
 1164718,
 1164719,
 1164720,
 1164721,
 1164722,
 1164723,
 1164724,
 1164725,
 1164726,
 1164727,
 1164728,
 1164729,
 1164730,
 1164731,
 1164732,
 1164733,
 1164734,
 1164735,
 1164736,
 1164737,
 1164738,
 1164739,
 1164740,
 1164741,
 1164742,
 1164743,
 1164744,
 1164745,
 1164746,
 1164747,
 1164748,
 1164749,
 1164750,
 1164751,
 1164752,
 1164753,
 1164754,
 1164755,
 1164756,
 1164757,
 1164758,
 1164759,
 1164760,
 1164761,
 1164762,
 1164763,
 1164764,
 1164765,
 1164766,
 1164767,
 1164768,
 1164769,
 1164770,
 1164771,
 1164772,
 1164773,
 1164774,
 1164775,
 1164776,
 1164777,
 1164778,
 1164779,
 1164780,
 1164781,
 1164782,
 1164783,
 1164784,
 1164785,
 1164786,
 1164787,
 1164788,
 1164789,
 1164790,
 1164791,
 1164792,
 1164793,
 1164794,
 1164795,
 1164796,
 1164797,
 1164798,
 1164799]

In [47]:
read.reference_start

1164707

In [48]:
states = {}
read_index = 0


In [58]:
cpg_table

0->0  0->1  1->0  1->1  0  1
chromosome start   end                                  
chr1       1164696 1164697     1     0     0     0  1  0
           1164710 1164711     1     0     0     0  3  0
           1164727 1164728     2     1     0     0  2  1
           1164732 1164733     2     0     0     1  2  1
           1164739 1164740     2     0     0     1  2  1
           1164741 1164742     2     0     0     1  2  1
           1164745 1164746     2     0     0     1  2  1
           1164775 1164776     2     0     0     1  2  1
           1164781 1164782     2     0     0     1  2  1
           1164689 1164690     0     0     0     0  1  0